## 如何引入pandas
下面的引用语句已经是惯例了。

In [1]:
import pandas as pd
import numpy as np

## DataFrame对象101
DataFrame对象由data表格数据，index行索引，columns列索引这三个主要的元素描述而成。

- `data` 接受的数据，比如python的列表或者numpy的ndarray对象等。
- `index`   行索引
- `columns`  列索引

In [2]:
data = np.arange(10)
data = data.reshape((5,2))
data

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [3]:
index = pd.date_range('2024/9/13', periods=5)
index

DatetimeIndex(['2024-09-13', '2024-09-14', '2024-09-15', '2024-09-16',
               '2024-09-17'],
              dtype='datetime64[ns]', freq='D')

In [4]:
columns = ['A', 'B']
df = pd.DataFrame(data=data, index=index, columns = columns)
df

,A,B
2024-09-13,0,1
2024-09-14,2,3
2024-09-15,4,5
2024-09-16,6,7
2024-09-17,8,9


In [5]:
df.index

DatetimeIndex(['2024-09-13', '2024-09-14', '2024-09-15', '2024-09-16',
               '2024-09-17'],
              dtype='datetime64[ns]', freq='D')

In [6]:
df.columns

Index(['A', 'B'], dtype='object')

In [7]:
df.dtypes

A    int64
B    int64
dtype: object

## DataFrame对象到numpy的ndarray对象
注意如果是df的部分列转成numpy对象如果各个列类型不一样的话那么是需要明确用 `dtype='what'` 来明确指定数据类型的。

In [8]:
df.to_numpy()

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

## 更改DataFrame某列的数据类型


In [9]:
df['A'] = df['A'].astype(float)
df.dtypes

A    float64
B      int64
dtype: object

## 获得某一列的值
注意返回的是Series对象

In [10]:
df['A']

2024-09-13    0.0
2024-09-14    2.0
2024-09-15    4.0
2024-09-16    6.0
2024-09-17    8.0
Freq: D, Name: A, dtype: float64

## 获得某一行的值
注意返回的是Series对象。

如果行和列都有有意义的名字，那么推荐使用 `loc` ，否则就需要使用 `iloc` 。下面没有演示iloc，但大体语法类似，比如 `iloc[0,1]` 就是0行1列。 

In [11]:
df.loc['2024-9-13']

A    0.0
B    1.0
Name: 2024-09-13 00:00:00, dtype: float64

## 获取某行某列的值
多行多列就是在下面语法的基础上增加 `:` 表达。

In [12]:
df.loc['2024-9-13', 'B']

np.float64(1.0)

## Series对象101

- `data` 接受的数据
- `index` 行索引

In [13]:
df['A'].to_numpy()

array([0., 2., 4., 6., 8.])

In [14]:
df['A'].index

DatetimeIndex(['2024-09-13', '2024-09-14', '2024-09-15', '2024-09-16',
               '2024-09-17'],
              dtype='datetime64[ns]', freq='D')

## DataFrame增加一列


In [15]:
df['C'] = 12
df

,A,B,C
2024-09-13,0.0,1,12
2024-09-14,2.0,3,12
2024-09-15,4.0,5,12
2024-09-16,6.0,7,12
2024-09-17,8.0,9,12


DataFrame某一列基于其他列的运算而获得

In [16]:
df['D'] = df['B'].map(lambda item: item+2)
df

,A,B,C,D
2024-09-13,0.0,1,12,3
2024-09-14,2.0,3,12,5
2024-09-15,4.0,5,12,7
2024-09-16,6.0,7,12,9
2024-09-17,8.0,9,12,11


## pandas的io接口
pandas的io接口提供了强大的文件读写支持，推荐使用，具体请参看官方文档。 [https://pandas.pydata.org/docs/user_guide/io.html](https://pandas.pydata.org/docs/user_guide/io.html)

In [17]:
example_csv_content = """
NumRooms,RoofType,Price
,,127500
2,,106000
4,Slate,178100
,,140000
"""
from io import StringIO
example_csv = StringIO(example_csv_content)

In [18]:
data = pd.read_csv(example_csv)
data

,NumRooms,RoofType,Price
0,NaN,NaN,127500
1,2.0,NaN,106000
2,4.0,Slate,178100
3,NaN,NaN,140000


## 处理缺少值
- `pd.get_dummies` 将分类变量转成虚拟变量，可以用 `dummy_na=True` 让缺少值的分类另外单独起一列。
- `DataFrame.dropna()` 删除缺少值的行和列
- `DataFrame.fillna(value)` 用某个值来填充缺少值的单元格 

In [19]:
inputs, targets = data.iloc[:, 0:-1], data.iloc[:, -1]
inputs

,NumRooms,RoofType
0,NaN,NaN
1,2.0,NaN
2,4.0,Slate
3,NaN,NaN


In [20]:
inputs = pd.get_dummies(inputs, dummy_na=True)
inputs

,NumRooms,RoofType_Slate,RoofType_nan
0,NaN,False,True
1,2.0,False,True
2,4.0,True,False
3,NaN,False,True


In [21]:
inputs.dtypes

NumRooms          float64
RoofType_Slate       bool
RoofType_nan         bool
dtype: object

In [22]:
inputs = inputs.fillna(inputs.mean())
inputs

,NumRooms,RoofType_Slate,RoofType_nan
0,3.0,False,True
1,2.0,False,True
2,4.0,True,False
3,3.0,False,True
